In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from xgboost import XGBRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/neolen-house-price-prediction/sample_submission.csv
/kaggle/input/neolen-house-price-prediction/data_description.txt
/kaggle/input/neolen-house-price-prediction/train.csv
/kaggle/input/neolen-house-price-prediction/test.csv
/kaggle/input/neolen-house-price-prediction/house-prices-advanced-regression-techniques.zip


In [2]:
# Read csv files. And keep a copy of the original dataframes. tran_df and test_df will be used throughout this notebook for preprocessing and scaling.we need original test dataframe in the end 
or_train_df = pd.read_csv('/kaggle/input/neolen-house-price-prediction/train.csv')
or_test_df = pd.read_csv('/kaggle/input/neolen-house-price-prediction/test.csv')

train_df = or_train_df
test_df = or_test_df

In [3]:
# Dropping 'Id' column as it has no use in training
train_df = train_df.drop(['Id'], axis=1)
test_df = test_df.drop(['Id'], axis=1)

In [4]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1259, 80)
Shape of test set: (201, 79)


## Replace NaNs
**While replacing we have to make sure to replace it with '0' (character) in case of a string column and 0 (number) if otherwise**

In [5]:
str_columns = []
num_columns = []

def convert_to_str(df):
    for col in df.columns:
        if (df[col].dtype == np.int64 or df[col].dtype == np.float64):
            df[col] = df[col].fillna(method='ffill')
            if col not in num_columns:
                num_columns.append(col)
        else:
            df[col] = df[col].fillna(method='ffill')
            if col not in str_columns:
                str_columns.append(col)
    return df

train_df = convert_to_str(train_df)
test_df = convert_to_str(test_df)

In [6]:
train_df['type'] = 'train'
test_df['type'] = 'test'

# Add a dummy SalePrice column to test dataframe to make number of columns equal 
test_df['SalePrice'] = train_df['SalePrice'].iloc[:201]

df = train_df.append(test_df, ignore_index=True)

In [7]:
for col in str_columns:
    one_hot = pd.get_dummies(df[col])

    replace_cols = {}
    for one_col in one_hot.columns:
        replace_cols[one_col] = f"{col}_{one_col}"
    one_hot = one_hot.rename(columns=replace_cols)

    df = df.drop(col, axis = 1)
    df = df.join(one_hot)

In [8]:
train_df = df[df['type'] == 'train']
test_df = df[df['type'] == 'test']

train_df = train_df.drop(['type'], axis=1)
test_df = test_df.drop(['type'], axis=1)

test_df = test_df.reset_index(drop=True)

In [9]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1259, 289)
Shape of test set: (201, 289)


## Standardization of values

In [10]:
scaler = StandardScaler()
scaler.fit(train_df[num_columns])

StandardScaler()

In [11]:
train_df[num_columns] = scaler.transform(train_df[num_columns])
test_df[num_columns] = scaler.transform(test_df[num_columns])

In [12]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1259, 289)
Shape of test set: (201, 289)


In [13]:
test_df = test_df.drop(['SalePrice'], axis=1)

In [14]:
train_labels = train_df['SalePrice']
train_data = train_df.drop(['SalePrice'], axis=1)

## XGBoost regressor model 

In [15]:
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [16]:
model.fit(train_data, train_labels)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
predictions = model.predict(test_df)

In [18]:
test_df['SalePrice'] = predictions
test_df[num_columns] = scaler.inverse_transform(test_df[num_columns])
test_df = pd.DataFrame(test_df, columns=train_df.columns)

In [19]:
# Create results dataframe
results = pd.DataFrame()
results['Id'] = or_test_df['Id']
results['SalePrice'] = test_df['SalePrice']

In [20]:
results.head()

,Id,SalePrice
0,1260,149337.478288
1,1261,181200.676521
2,1262,135289.445715
3,1263,129913.254521
4,1264,161978.788920


In [21]:
results.to_csv('submissions.csv', index=False)